<a href="https://colab.research.google.com/github/gbiamgaurav/Feature-Selection/blob/main/Chi_Square_Test_for_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fisher Score- Chisquare Test For Feature Selection
Compute chi-squared stats between each non-negative feature and class.

This score should be used to evaluate categorical variables in a classification task.
This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [3]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.dtypes

survived          int64
pclass            int64
sex              object
age             float64
sibsp             int64
parch             int64
fare            float64
embarked         object
class          category
who              object
adult_male         bool
deck           category
embark_town      object
alive            object
alone              bool
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
###['sex', 'embarked', 'alone', 'pclass', 'Survived']

In [7]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [8]:
df = df[['sex', 'embarked', 'alone', 'pclass', 'survived']]

In [9]:
df.head()

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1
3,female,S,False,1,1
4,male,S,True,3,0


In [10]:
### Lets perform encoding

df['sex'] = pd.factorize(df.sex)[0]
df['alone'] = pd.factorize(df.alone)[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
df.head()

,sex,embarked,alone,pclass,survived
0,0,S,0,3,0
1,1,C,0,1,1
2,1,S,1,3,1
3,1,S,0,1,1
4,0,S,1,3,0


In [12]:
df.sex.unique()

array([0, 1])

In [13]:
df.sex.value_counts()

0    577
1    314
Name: sex, dtype: int64

In [14]:
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [15]:
df.isnull().sum()

sex         0
embarked    2
alone       0
pclass      0
survived    0
dtype: int64

In [16]:
df.pclass.value_counts()

3    491
1    216
2    184
Name: pclass, dtype: int64

In [23]:
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [24]:
df.embarked.isnull().sum()

2

In [26]:
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [27]:
df.embarked.isnull().sum()

2

In [29]:
### let's perform label encoding on embarked
label = {k: i for i, k in enumerate(df['embarked'].unique(), 0)}
df['embarked'] = df['embarked'].map(label)

In [30]:
df.head()

,sex,embarked,alone,pclass,survived
0,0,0,0,3,0
1,1,1,0,1,1
2,1,0,1,3,1
3,1,0,0,1,1
4,0,0,1,3,0


In [31]:
df.isnull().sum()

sex         0
embarked    0
alone       0
pclass      0
survived    0
dtype: int64

In [34]:
df.embarked.isnull().sum()

0

In [32]:
### train test split is usually done to avoid overfitting

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['sex', 'embarked', 'alone', 'pclass']], df['survived'],
                                                    test_size=0.3,
                                                    random_state=100)

In [33]:
X_train.head()

,sex,embarked,alone,pclass
69,0,0,0,3
85,1,0,0,3
794,0,0,1,3
161,1,0,1,2
815,0,0,1,1


In [19]:
X_train['sex'].unique()

array([0, 1])

In [20]:
X_train.isnull().sum()

sex         0
embarked    1
alone       0
pclass      0
dtype: int64

In [35]:
### Perform Chi-Square Test
### chi2 return 2 values: 
### F-Score and p-value
from sklearn.feature_selection import chi2
f_p_values = chi2(X_train, y_train)

In [36]:
f_p_values

(array([120.31252684,   7.55053653,  10.88471585,  21.97994154]),
 array([5.40398040e-28, 5.99922095e-03, 9.69610546e-04, 2.75514881e-06]))

In [37]:
p_values = pd.Series(f_p_values[1])
p_values.index = X_train.columns
p_values

sex         5.403980e-28
embarked    5.999221e-03
alone       9.696105e-04
pclass      2.755149e-06
dtype: float64

In [41]:
p_values.sort_index(ascending=False)

sex         5.403980e-28
pclass      2.755149e-06
embarked    5.999221e-03
alone       9.696105e-04
dtype: float64

### Observation :

### `Sex` column is the most important column when compared to the output feature `survived`